In [1]:
# Imports section
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
# Read the file in the format given
text=""
nLines=0
nAttributes=0
nClasses=0
attributesName = []
data = []
dataPerAttribute = []
classes = []
try:
    with open("archivo.txt","r") as file:
        count=0
        for line in file:
            if count < 3:
                if count == 0:
                    nLines = int(line.strip())
                else:
                    if count == 1:
                        nAttributes = int(line.strip())
                        for i in range(1,nAttributes+1):
                            attributesName.append("att"+str(i))
                            dataPerAttribute.append([])
                    else:
                        if count == 2:
                            nClasses = int(line.strip())
            else:
                split_string_S = line.strip().split(',')
                
                count_split = 0
                split_string_n = []
                for split in split_string_S:
                    if count_split >= nAttributes:
                        split_string_n.append(int(split))
                    else:
                        split_string_n.append(float(split))
                    count_split += 1
                    
                data.append(split_string_n)
                classes.append(split_string_n[nAttributes])
                n_attribute = 0
                for attribute in split_string_n:
                    if n_attribute >= nAttributes:
                        break
                    else:
                        dataPerAttribute[n_attribute].append(attribute)
                        n_attribute += 1
            count += 1
        print("EOF reached")
except FileNotFoundError:
    text="Archivo no existe"
    exit()
finally:
    file.close()
    # print(str(nLines) + "\n")
    # print(str(nAttributes) + "\n")
    # print(str(nClasses) + "\n")
    # print(attributesName)
    # print(data)

EOF reached


In [3]:
# Turning the data into a DataFrame python object
columns_ = attributesName[:]
columns_.append("target")

dataFrame = pd.DataFrame(data=data, columns=columns_)
dataFrame

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,target
0,140.0,125.0,0.0,0.0,0.000000,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818,0
1,188.0,133.0,0.0,0.0,0.000000,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817,0
2,105.0,139.0,0.0,0.0,0.000000,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946,0
3,34.0,137.0,0.0,0.0,0.000000,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272,0
4,39.0,111.0,0.0,0.0,0.000000,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,225.0,58.0,0.0,0.0,0.000000,0.333334,0.421637,0.444445,0.344265,8.333333,5.555555,14.111111,5.333334,-8.333333,17.333334,-9.000000,14.111111,0.622222,-2.068564,6
416,170.0,154.0,0.0,0.0,0.000000,0.111111,0.172133,0.055556,0.136083,0.074074,0.000000,0.222222,0.000000,-0.222222,0.444444,-0.222222,0.222222,0.222222,-2.094395,6
417,238.0,61.0,0.0,0.0,0.000000,0.666667,0.471405,0.722222,0.490653,8.777778,5.888889,14.333333,6.111111,-8.666667,16.666666,-8.000000,14.333333,0.588823,-2.121837,6
418,189.0,62.0,0.0,0.0,0.000000,0.388889,0.250924,0.444444,0.344265,7.629630,5.000000,13.111111,4.777778,-7.888889,16.444445,-8.555555,13.111111,0.635938,-2.066953,6


In [4]:
# Separating out the features
x = dataFrame.loc[:, attributesName].values
# Separating out the target
y = dataFrame.loc[:,['target']].values


In [5]:
def get_score(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train.ravel())
    return model.score(x_test,y_test)
folds = int(input("Enter number of K-Folds for Cross Validation: "))
neighbors = int(input("Enter value of K-Nearest Neighbors: "))
kk = KNeighborsClassifier(n_neighbors=neighbors)
nb = GaussianNB()
hLayers = int(input("Enter number of Hidden Layers for the Neural Network: "))
layersSize=[]
for i in range(1,hLayers+1):
    layersSize.append(int(input("Number of perceptrons for the "+str(i)+"-th layer: ")))
activationFunction=input("activation{identity, logistic, tanh, relu}: ")
epochs = int(input("Number of iterations for the Neural Network: "))
moment = float(input("Momentum for the Neural Network[0-1]: "))
nn = MLPClassifier(hidden_layer_sizes=layersSize,solver='sgd',activation=activationFunction,max_iter=epochs,momentum=moment)
scores_KK = []
scores_NB = []
scores_NN = []
kk_df = []
nb_df = []
nn_df = []
avg_df = []
split_names = []
i=1
KFolds = StratifiedKFold(n_splits=folds)
for train_index,test_index in KFolds.split(X=x,y=y):
    X_train,X_test,y_train,y_test = x[train_index],x[test_index],y[train_index],y[test_index]
    scores_KK.append(get_score(kk,X_train,X_test,y_train,y_test))
    scores_NB.append(get_score(nb,X_train,X_test,y_train,y_test))
    scores_NN.append(get_score(nn,X_train,X_test,y_train,y_test))
    split_names.append("Fold "+str(i))
    i += 1
print("---------Stratified Cross Validation---------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df = pd.DataFrame(scores_KK,
                   columns=['Stratified Cross Validation'],
                     index=split_names)
nb_df = pd.DataFrame(scores_NB,
                   columns=['Stratified Cross Validation'],
                     index=split_names)
nn_df = pd.DataFrame(scores_NN,
                   columns=['Stratified Cross Validation'],
                     index=split_names)
strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df = pd.DataFrame(strat_avg,
                      columns=['Stratified Cross Validation'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
print("----Cross Validation Average----")
avg_df


Enter number of K-Folds for Cross Validation: 5
Enter value of K-Nearest Neighbors: 5
Enter number of Hidden Layers for the Neural Network: 3
Number of perceptrons for the 1-th layer: 10
Number of perceptrons for the 2-th layer: 15
Number of perceptrons for the 3-th layer: 10
activation{identity, logistic, tanh, relu}: tanh
Number of iterations for the Neural Network: 2500
Momentum for the Neural Network[0-1]: 0.9
---------Stratified Cross Validation---------
K-Nearest Neighbors
[0.8333333333333334, 0.8928571428571429, 0.9166666666666666, 0.9285714285714286, 0.9523809523809523]
Naive Bayer
[0.6428571428571429, 0.8809523809523809, 0.8928571428571429, 0.9166666666666666, 0.8571428571428571]
Neural Network
[0.5714285714285714, 0.8214285714285714, 0.8333333333333334, 0.8452380952380952, 0.8690476190476191]
----Cross Validation Average----


,Stratified Cross Validation
K-Neighbors,0.904762
Naive-Bayes,0.838095
Neural Network,0.788095


In [6]:
scores_KK = []
scores_NB = []
scores_NN = []
scores_KK = cross_val_score(kk, x, y.ravel(), cv=folds)
scores_NB = cross_val_score(nb, x, y.ravel(), cv=folds)
scores_NN = cross_val_score(nn, x, y.ravel(), cv=folds)
print("------Normal Cross Validation-------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df_aux = pd.DataFrame(scores_KK,
                   columns=['Normal Cross Validation'],
                         index=split_names)
kk_df_result = pd.concat([kk_df_aux,kk_df],axis=1)
kk_df = kk_df_result


nb_df_aux = pd.DataFrame(scores_NB,
                   columns=['Normal Cross Validation'],
                     index=split_names)
nb_df_result = pd.concat([nb_df_aux,nb_df],axis=1)
nb_df = nb_df_result


nn_df_aux = pd.DataFrame(scores_NN,
                   columns=['Normal Cross Validation'],
                         index=split_names)
nn_df_result = pd.concat([nn_df_aux,nn_df],axis=1)
nn_df = nn_df_result

strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df_aux = pd.DataFrame(strat_avg,
                      columns=['Normal Cross Validation'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
avg_df_result = pd.concat([avg_df_aux,avg_df],axis=1)
avg_df=avg_df_result
print("----Cross Validation Average----")
avg_df

------Normal Cross Validation-------
K-Nearest Neighbors
[0.83333333 0.89285714 0.91666667 0.92857143 0.95238095]
Naive Bayer
[0.64285714 0.88095238 0.89285714 0.91666667 0.85714286]
Neural Network
[0.70238095 0.80952381 0.76190476 0.77380952 0.76190476]
----Cross Validation Average----


,Normal Cross Validation,Stratified Cross Validation
K-Neighbors,0.904762,0.904762
Naive-Bayes,0.838095,0.838095
Neural Network,0.761905,0.788095


In [7]:
# Standardizing the features
x = StandardScaler().fit_transform(x)
scores_KK = []
scores_NB = []
scores_NN = []
KFolds = StratifiedKFold(n_splits=folds)
for train_index,test_index in KFolds.split(X=x,y=y):
    X_train,X_test,y_train,y_test = x[train_index],x[test_index],y[train_index],y[test_index]
    scores_KK.append(get_score(kk,X_train,X_test,y_train,y_test))
    scores_NB.append(get_score(nb,X_train,X_test,y_train,y_test))
    scores_NN.append(get_score(nn,X_train,X_test,y_train,y_test))
print("------Stratified Cross Validation with Standardized data-------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df_aux = pd.DataFrame(scores_KK,
                   columns=['Stratified Cross Validation w/SD'],
                         index=split_names)
kk_df_result = pd.concat([kk_df_aux,kk_df],axis=1)
kk_df = kk_df_result


nb_df_aux = pd.DataFrame(scores_NB,
                   columns=['Stratified Cross Validation w/SD'],
                     index=split_names)
nb_df_result = pd.concat([nb_df_aux,nb_df],axis=1)
nb_df = nb_df_result


nn_df_aux = pd.DataFrame(scores_NN,
                   columns=['Stratified Cross Validation w/SD'],
                         index=split_names)
nn_df_result = pd.concat([nn_df_aux,nn_df],axis=1)
nn_df = nn_df_result

strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df_aux = pd.DataFrame(strat_avg,
                      columns=['Stratified Cross Validation w/SD'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
avg_df_result = pd.concat([avg_df_aux,avg_df],axis=1)
avg_df=avg_df_result
print("----Stratified Cross Validation w/SD average----")
avg_df

------Stratified Cross Validation with Standardized data-------
K-Nearest Neighbors
[0.8809523809523809, 0.8809523809523809, 0.9404761904761905, 0.9285714285714286, 0.9285714285714286]
Naive Bayer
[0.6071428571428571, 0.7857142857142857, 0.8809523809523809, 0.9166666666666666, 0.8333333333333334]
Neural Network
[0.8928571428571429, 0.9166666666666666, 0.9166666666666666, 0.9285714285714286, 0.9404761904761905]
----Stratified Cross Validation w/SD average----


,Stratified Cross Validation w/SD,Normal Cross Validation,Stratified Cross Validation
K-Neighbors,0.911905,0.904762,0.904762
Naive-Bayes,0.804762,0.838095,0.838095
Neural Network,0.919048,0.761905,0.788095


In [8]:
scores_KK = []
scores_NB = []
scores_NN = []
scores_KK = cross_val_score(kk, x, y.ravel(), cv=folds)
scores_NB = cross_val_score(nb, x, y.ravel(), cv=folds)
scores_NN = cross_val_score(nn, x, y.ravel(), cv=folds)
print("------Normal Cross Validation with Standardized data-------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df_aux = pd.DataFrame(scores_KK,
                   columns=['Normal Cross Validation w/SD'],
                         index=split_names)
kk_df_result = pd.concat([kk_df_aux,kk_df],axis=1)
kk_df = kk_df_result


nb_df_aux = pd.DataFrame(scores_NB,
                   columns=['Normal Cross Validation w/SD'],
                     index=split_names)
nb_df_result = pd.concat([nb_df_aux,nb_df],axis=1)
nb_df = nb_df_result


nn_df_aux = pd.DataFrame(scores_NN,
                   columns=['Normal Cross Validation w/SD'],
                         index=split_names)
nn_df_result = pd.concat([nn_df_aux,nn_df],axis=1)
nn_df = nn_df_result

strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df_aux = pd.DataFrame(strat_avg,
                      columns=['Normal Cross Validation w/SD'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
avg_df_result = pd.concat([avg_df_aux,avg_df],axis=1)
avg_df=avg_df_result
print("----Normal Cross Validation w/SD average----")
avg_df

------Normal Cross Validation with Standardized data-------
K-Nearest Neighbors
[0.88095238 0.88095238 0.94047619 0.92857143 0.92857143]
Naive Bayer
[0.60714286 0.78571429 0.88095238 0.91666667 0.83333333]
Neural Network
[0.9047619  0.9047619  0.89285714 0.9047619  0.94047619]
----Normal Cross Validation w/SD average----


,Normal Cross Validation w/SD,Stratified Cross Validation w/SD,Normal Cross Validation,Stratified Cross Validation
K-Neighbors,0.911905,0.911905,0.904762,0.904762
Naive-Bayes,0.804762,0.804762,0.838095,0.838095
Neural Network,0.909524,0.919048,0.761905,0.788095


In [9]:
# PCA
pcn = int(input('Number of principal components:'))
if pcn > nAttributes:
    print(str(nAttributes) + " is the maximum number of principal components")
    pcn = min([nAttributes,pcn])
else:
    if pcn <= 0:
        print("1 is the minimum number of principal components")
        pcn = max([1,pcn])
        
print ("Creating PCA with " + str(pcn) + " components")
pca = PCA(n_components=pcn)
principalComponents = pca.fit_transform(x)
principalComponents

Number of principal components:10
Creating PCA with 10 components


array([[-2.29967726, -0.34383656, -1.03621076, ...,  0.26709305,
        -0.33448557,  0.11302275],
       [-2.3719252 , -0.40386692, -1.03571738, ...,  0.20000508,
        -0.31410749,  0.1010035 ],
       [-2.38958998, -0.26657193, -0.77018467, ...,  0.23857003,
        -0.46191418,  0.10351236],
       ...,
       [-1.26213535, -0.47888842, -2.06686476, ...,  0.18859249,
         0.2628445 ,  0.1196399 ],
       [-1.40324394, -0.49411006, -2.02419759, ...,  0.19228983,
         0.0543304 ,  0.0832061 ],
       [-0.2188762 ,  1.68669316, -1.76560626, ...,  1.24586607,
         0.59707966,  1.27070754]])

In [10]:
scores_KK = []
scores_NB = []
scores_NN = []
KFolds = StratifiedKFold(n_splits=folds)
for train_index,test_index in KFolds.split(X=x,y=y):
    X_train,X_test,y_train,y_test = principalComponents[train_index],principalComponents[test_index],y[train_index],y[test_index]
    scores_KK.append(get_score(kk,X_train,X_test,y_train,y_test))
    scores_NB.append(get_score(nb,X_train,X_test,y_train,y_test))
    scores_NN.append(get_score(nn,X_train,X_test,y_train,y_test))
print("------Stratified Cross Validation with PCA-------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df_aux = pd.DataFrame(scores_KK,
                   columns=['Stratified Cross Validation with PCA'],
                         index=split_names)
kk_df_result = pd.concat([kk_df_aux,kk_df],axis=1)
kk_df = kk_df_result


nb_df_aux = pd.DataFrame(scores_NB,
                   columns=['Stratified Cross Validation with PCA'],
                     index=split_names)
nb_df_result = pd.concat([nb_df_aux,nb_df],axis=1)
nb_df = nb_df_result


nn_df_aux = pd.DataFrame(scores_NN,
                   columns=['Stratified Cross Validation with PCA'],
                         index=split_names)
nn_df_result = pd.concat([nn_df_aux,nn_df],axis=1)
nn_df = nn_df_result

strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df_aux = pd.DataFrame(strat_avg,
                      columns=['Stratified Cross Validation with PCA'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
avg_df_result = pd.concat([avg_df_aux,avg_df],axis=1)
avg_df=avg_df_result
print("----Stratified Cross Validation with PCA average----")
avg_df

------Stratified Cross Validation with PCA-------
K-Nearest Neighbors
[0.8571428571428571, 0.8809523809523809, 0.9285714285714286, 0.9047619047619048, 0.9285714285714286]
Naive Bayer
[0.7380952380952381, 0.9047619047619048, 0.9404761904761905, 0.8452380952380952, 0.8809523809523809]
Neural Network
[0.8333333333333334, 0.9047619047619048, 0.8690476190476191, 0.9166666666666666, 0.9047619047619048]
----Stratified Cross Validation with PCA average----


,Stratified Cross Validation with PCA,Normal Cross Validation w/SD,Stratified Cross Validation w/SD,Normal Cross Validation,Stratified Cross Validation
K-Neighbors,0.900000,0.911905,0.911905,0.904762,0.904762
Naive-Bayes,0.861905,0.804762,0.804762,0.838095,0.838095
Neural Network,0.885714,0.909524,0.919048,0.761905,0.788095


In [11]:
scores_KK = []
scores_NB = []
scores_NN = []
scores_KK = cross_val_score(kk, principalComponents, y.ravel(), cv=folds)
scores_NB = cross_val_score(nb, principalComponents, y.ravel(), cv=folds)
scores_NN = cross_val_score(nn, principalComponents, y.ravel(), cv=folds)
print("------Normal Cross Validation with PCA-------")
print("K-Nearest Neighbors")
print(scores_KK)
print("Naive Bayer")
print(scores_NB)
print("Neural Network")
print(scores_NN)
kk_df_aux = pd.DataFrame(scores_KK,
                   columns=['Normal Cross Validation with PCA'],
                         index=split_names)
kk_df_result = pd.concat([kk_df_aux,kk_df],axis=1)
kk_df = kk_df_result


nb_df_aux = pd.DataFrame(scores_NB,
                   columns=['Normal Cross Validation with PCA'],
                     index=split_names)
nb_df_result = pd.concat([nb_df_aux,nb_df],axis=1)
nb_df = nb_df_result


nn_df_aux = pd.DataFrame(scores_NN,
                   columns=['Normal Cross Validation with PCA'],
                         index=split_names)
nn_df_result = pd.concat([nn_df_aux,nn_df],axis=1)
nn_df = nn_df_result

strat_avg = [np.mean(scores_KK),
             np.mean(scores_NB),
             np.mean(scores_NN)]
avg_df_aux = pd.DataFrame(strat_avg,
                      columns=['Normal Cross Validation with PCA'],index=['K-Neighbors','Naive-Bayes','Neural Network'])
avg_df_result = pd.concat([avg_df_aux,avg_df],axis=1)
avg_df=avg_df_result
print("----Normal Cross Validation with PCA average----")
avg_df

------Normal Cross Validation with PCA-------
K-Nearest Neighbors
[0.85714286 0.88095238 0.92857143 0.9047619  0.92857143]
Naive Bayer
[0.73809524 0.9047619  0.94047619 0.8452381  0.88095238]
Neural Network
[0.88095238 0.92857143 0.89285714 0.95238095 0.91666667]
----Normal Cross Validation with PCA average----


,Normal Cross Validation with PCA,Stratified Cross Validation with PCA,Normal Cross Validation w/SD,Stratified Cross Validation w/SD,Normal Cross Validation,Stratified Cross Validation
K-Neighbors,0.900000,0.900000,0.911905,0.911905,0.904762,0.904762
Naive-Bayes,0.861905,0.861905,0.804762,0.804762,0.838095,0.838095
Neural Network,0.914286,0.885714,0.909524,0.919048,0.761905,0.788095
